# Subgraphs

Useful in creating multi agent systems.

There are 2 ways how to utilize subgraphs:

## 1. Register subgraph as a node

```
builder.add_node("sub_graph", graph_builder.compile())
```

## 2. Invoke a subgraph from a node

```
def sub_graph_node(state: State) -> State:
    sub_graph.invoke({"question": "should i invest in AI stocks now"})
```

In [122]:
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display
from typing_extensions import TypedDict
from typing import Annotated, List
import operator
from langgraph.graph import StateGraph, START, END

from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools import TavilySearchResults


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

![Subgraphs](images/subgraphs.png)

## Optimized Web Search Graph

In [ ]:
from langchain_community.tools import TavilySearchResults

class OptimizedWebSearchState(TypedDict):
    question: List[str]
    classification: List[str]
    optimized_web_request: str
    documents: List[str]


def optimize_request(state: OptimizedWebSearchState):
    system_message = SystemMessage(content=("""
        You are a query optimization assistant.  
        You receive two inputs:  
            - `category`: The predefined category of the query.  
            - `user_request`: The original user query.  

        Your task is to refine user queries for better search results while ensuring 
        relevance to the given category. Keep the original intent but improve clarity, 
        specificity, and structure.  

        - **Do not return a short, vague phrase.** The query must be structured for search engines to retrieve high-quality results.  
        - **Convert questions into search-friendly statements.** Do not return a question.  
        - **Ensure the query is fact-based**—remove subjective or vague phrasing like “should I,” “is it a good idea,” or “viable option.”  
        - **Use the category to improve relevance** but do not assume unnecessary details.  
        - **Never add a specific year unless the user explicitly includes one.** If a time reference is needed, use ‘now’ instead.  
        - **If the query is broad, narrow it down** by adding relevant **context** (e.g., key factors, market trends, risks, industries).  
        - **Avoid single-word or ultra-short responses.** The optimized query must be **concise but meaningful**.  

        Return **only the optimized query as plain text**, with no explanations, formatting, or extra text—just the improved query.
    """))

    human_message = HumanMessage(content=(f"""
        category: {state["classification"][0]}

        user_request: {state["question"][0]}
    """))
    
    optimized_web_request = llm.invoke([system_message, human_message])

    return {"optimized_web_request": optimized_web_request.content}


def search_web(state: OptimizedWebSearchState):
    tavily_search = TavilySearchResults(max_results=10)
    search_docs = tavily_search.invoke(state['optimized_web_request'])

    # Keywords that indicate an error or access restriction
    forbidden_keywords = [
        "403 Forbidden", "Access denied", "CAPTCHA", 
        "has been denied", "not authorized", "verify you are a human"
    ]

    results = [
        f'<Document url="{doc["url"]}">\n{doc["content"]}\n</Document>'
        for doc in search_docs
        if not any(keyword.lower() in doc["content"].lower() for keyword in forbidden_keywords)
    ]

    return {"documents": results}


optimized_web_search_builder = StateGraph(OptimizedWebSearchState)
optimized_web_search_builder.add_node(optimize_request)
optimized_web_search_builder.add_node(search_web)

optimized_web_search_builder.add_edge(START, "optimize_request")
optimized_web_search_builder.add_edge("optimize_request", "search_web")
optimized_web_search_builder.add_edge("search_web", END)

graph = optimized_web_search_builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({
    "question": ["should i invest in AI stocks now"],
    "classification": ["Investment Advice"]
})
result

## Structured Wikipedia Lookup Graph

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

class StructuredWikipediaLookupState(TypedDict):
    question: List[str]
    classification: List[str]
    optimized_structured_wikipedia: str
    documents: List[str]


def optimize_request(state: StructuredWikipediaLookupState):
    system_message = SystemMessage(content=("""
        You optimize queries for searching Wikipedia articles.  
        You receive two inputs: 
            - `category`: The predefined category of the query. 
            - `user_request`: The original user query. 

        Use the **category** to refine the query while ensuring it aligns with Wikipedia's factual, encyclopedia-style content.  

            - **Never frame the query as a question.** Convert it into a Wikipedia-style article title.  
            - **Avoid subjective, speculative, or opinion-based wording.** Focus on factual, well-defined topics.  
            - **Ensure the optimized query is informative and structured as an encyclopedia entry.**  
            - **Use the category to guide query refinement** but do **not assume** or limit responses to a specific domain.  
            - **If the category suggests an advisory or decision-making topic (e.g., "Investment Advice"), shift the focus to established concepts, historical context, or factual analysis.**  

        Return **only the optimized query as plain text**, with no explanations, formatting, or extra text—just the improved query.
    """))

    human_message = HumanMessage(content=(f"""
        category: {state["classification"][0]}

        user_request: {state["question"][0]}
    """))
    
    optimized_structured_wikipedia = llm.invoke([system_message, human_message])

    return {"optimized_structured_wikipedia": optimized_structured_wikipedia.content}


def search_wikipedia(state: StructuredWikipediaLookupState):
    search_docs = WikipediaLoader(query=state['optimized_structured_wikipedia'], load_max_docs=10).load()

    results = [
        f'<Document url="{doc.metadata["source"]}">\n{doc.page_content}\n</Document>'
        for doc in search_docs
    ]
    
    return {"documents": results} 


structured_wikipedia_lookup_builder = StateGraph(StructuredWikipediaLookupState)
structured_wikipedia_lookup_builder.add_node(optimize_request)
structured_wikipedia_lookup_builder.add_node(search_wikipedia)

structured_wikipedia_lookup_builder.add_edge(START, "optimize_request")
structured_wikipedia_lookup_builder.add_edge("optimize_request", "search_wikipedia")
structured_wikipedia_lookup_builder.add_edge("search_wikipedia", END)

graph = structured_wikipedia_lookup_builder.compile()

display(Image(structured_wikipedia_lookup.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({
    "question": ["should i invest in AI stocks now"],
    "classification": ["Investment Advice"]
})
result

## Entire Graph

In [ ]:
import operator

class EntireGraphState(TypedDict):
    question: Annotated[list, operator.add]
    classification: Annotated[list, operator.add]
    documents: Annotated[list, operator.add]
    answer: str


def query_classification(state: EntireGraphState):
    system_message = SystemMessage(content=("""
        You are a query classification assistant. Your task is to analyze user queries and assign 
        them to the most relevant category. Do not force-fit queries into predefined categories—only 
        use an existing category if it is a clear match. Otherwise, create a new category that 
        accurately represents the topic.
	        - Assign a category based on the core subject of the query.
	        - If the query clearly fits an existing category, use it.
	        - If no existing category is a perfect match, create a new descriptive category.
	        - Do not return ‘unknown’—always generate a meaningful category name.
	    
        Return only the category name as plain text, without explanations or extra text.
    """))

    classification = llm.invoke([system_message, HumanMessage(content=state["question"][0])])

    return {"classification": [classification.content]}


def generate_answer(state: EntireGraphState):
    # System message
    system_message = SystemMessage(content=("""
        You are an AI assistant that answers questions based on the provided documents.  

        ### **Guidelines:**  
            - Provide **direct, concise, and accurate** answers.  
            - When possible, **cite the relevant document or URL** next to the information used.  
            - If multiple documents contain relevant information, **synthesize the best answer** while ensuring clarity.  
            - If a document contains **conflicting information**, mention both perspectives.  
            - **When using a specific document for a prediction or key statement, include its source URL immediately after the referenced information** so the user can verify details.
    """))


    formatted_docs = "\n".join(
        [f"- {doc}" for doc in state["documents"]]
    )

    system_context = SystemMessage(content=(f"Use the following documents as context for your response:\n\n{formatted_docs}"))

    answer = llm.invoke([system_message, system_context, HumanMessage(content=state["question"][0])])
    
    # Append it to state
    return {"answer": answer}


builder = StateGraph(EntireGraphState)
builder.add_node(query_classification)
builder.add_node("optimized_web_search", optimized_web_search_builder.compile())
builder.add_node("structured_wikipedia_lookup", structured_wikipedia_lookup_builder.compile())
builder.add_node(generate_answer)

builder.add_edge(START, "query_classification")
builder.add_edge("query_classification", "optimized_web_search")
builder.add_edge("query_classification", "structured_wikipedia_lookup")
builder.add_edge("optimized_web_search", "generate_answer")
builder.add_edge("structured_wikipedia_lookup", "generate_answer")
builder.add_edge("generate_answer", END)

graph = builder.compile()

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": ["should i invest in AI stocks now"]})
result

In [ ]:
print(result["answer"].content)